In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/Users/fa/.config/gcloud/application_default_credentials.json' #(path to you keep in step 5 )
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

## Number of ICU Transfer Events within 52 hrs of alert

In [9]:
%%bigquery df
WITH random_flag AS (
  SELECT 
    anon_id, 
    pat_enc_csn_id_coded, 
    smrtdta_elem_value AS random_flag 
  FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_smrtdta` 
  WHERE concept_id = 'SHC#6051'
),

discharges_selected AS (
  SELECT 
    anon_id, 
    pat_enc_csn_id_coded, 
    effective_time_jittered, 
    department_name 
  FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt`
  INNER JOIN `som-nero-phi-jonc101-secure.shc_core_updates.shc_dep_map` USING(department_id)
  INNER JOIN `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-04-15` USING (anon_id)
  WHERE 
    event_type = 'Discharge' 
    AND (
      UPPER(department_name) LIKE 'B3' OR
      UPPER(department_name) LIKE 'C3' OR
      UPPER(department_name) LIKE 'M7' OR
      UPPER(department_name) LIKE 'L7' OR
      UPPER(department_name) LIKE '1%WEST%' OR
      UPPER(department_name) LIKE '2%NORTH%' OR
      UPPER(department_name) LIKE '2%WEST%' OR
      UPPER(department_name) LIKE '3%WEST%'
    )
    AND effective_time_jittered - INTERVAL jitter DAY BETWEEN '2024-08-15' AND '2025-03-15'
),

admissions AS (
  SELECT 
    anon_id, 
    pat_enc_csn_id_coded, 
    effective_time_jittered 
  FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt` 
  WHERE event_type = 'Admission'
),

hospital_stays_selected AS (
  SELECT 
    adm.anon_id, 
    adm.pat_enc_csn_id_coded, 
    adm.effective_time_jittered AS adm_time, 
    dis.effective_time_jittered AS dis_time, 
    dis.department_name,
    DATETIME_DIFF(dis.effective_time_jittered, adm.effective_time_jittered, DAY) + 1 AS duration,
    DATETIME_DIFF(dis.effective_time_jittered, adm.effective_time_jittered, MINUTE) / 60 AS los_hours
  FROM admissions adm 
  INNER JOIN discharges_selected dis USING (anon_id, pat_enc_csn_id_coded)
),

alerts_cbc AS (
  SELECT DISTINCT 
    alt.anon_id, 
    alt.pat_enc_csn_id_coded,
    alt.alt_id_coded, 
    alt.alert_desc, 
    his.update_date_jittered AS alt_time
  FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_alert` alt 
  INNER JOIN `som-nero-phi-jonc101-secure.shc_core_updates.shc_alert_history` his 
    USING (anon_id, alt_id_coded)
  INNER JOIN hospital_stays_selected USING (anon_id, pat_enc_csn_id_coded)
  WHERE alert_desc IN (
    'SHC AIML LAB CBC STABILITY BASE - LOUD PILOT', 
    'SHC AIML LAB CBC STABILITY BASE - SILENT'
  )
),

cohort AS (
  SELECT 
    alt.*, 
    rf.random_flag  
  FROM alerts_cbc alt 
  INNER JOIN random_flag rf USING (anon_id, pat_enc_csn_id_coded)
  INNER JOIN `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-04-15` USING (anon_id)
  WHERE NOT (
    alt.alert_desc = 'SHC AIML LAB CBC STABILITY BASE - SILENT' 
    AND rf.random_flag = '1'
  )
),
transfer_icu AS (
  SELECT 
    anon_id, 
    pat_enc_csn_id_coded, 
    effective_time_jittered AS icutime
  FROM `som-nero-phi-jonc101.shc_core_2024.adt`
  WHERE 
    pat_lv_of_care = 'Critical Care'
    AND event_type = 'Transfer In'
),

randomized_transfer_icu AS (
  SELECT 
    c.*, 
    t.icutime
  FROM cohort c 
  LEFT JOIN transfer_icu t 
    USING (anon_id, pat_enc_csn_id_coded)
    #WHERE  (icutime <= alt_time + INTERVAL 52 HOUR and random_flag='1') 
    
)
SELECT *
FROM randomized_transfer_icu


Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
df

,anon_id,pat_enc_csn_id_coded,alt_id_coded,alert_desc,alt_time,random_flag,icutime
0,JC3625330,131392377322,191333964,SHC AIML LAB CBC STABILITY BASE - LOUD PILOT,2024-12-12 10:07:00,1,NaT
1,JC6179169,131385167066,184537513,SHC AIML LAB CBC STABILITY BASE - LOUD PILOT,2024-08-09 10:06:00,1,NaT
2,JC3126288,131389819707,188742352,SHC AIML LAB CBC STABILITY BASE - LOUD PILOT,2024-09-24 10:05:00,1,NaT
3,JC6458077,131394266454,193207679,SHC AIML LAB CBC STABILITY BASE - LOUD PILOT,2024-12-24 16:04:00,1,NaT
4,JC746892,131393299173,192111438,SHC AIML LAB CBC STABILITY BASE - LOUD PILOT,2024-11-28 10:05:00,1,NaT
...,...,...,...,...,...,...,...
728,JC1670995,131393455679,193103383,SHC AIML LAB CBC STABILITY BASE - SILENT,2024-12-02 16:05:00,2,2024-11-24 02:58:00
729,JC1670995,131393455679,193103383,SHC AIML LAB CBC STABILITY BASE - SILENT,2024-12-02 16:05:00,2,2024-12-08 11:52:00
730,JC1670995,131393455679,193103383,SHC AIML LAB CBC STABILITY BASE - SILENT,2024-12-02 16:05:00,2,2024-12-04 07:51:00
731,JC2787315,131393048617,192008165,SHC AIML LAB CBC STABILITY BASE - SILENT,2024-11-13 10:06:00,2,NaT


In [18]:
df['alert_icu'] = ((df['icutime'] - df['alt_time']).dt.total_seconds() / 3600)
df['alert_icu'] = df['alert_icu'].where(df['alert_icu'].isnull(), df['alert_icu'].round().astype('Int64'))
print(f"Treatment:{df[(df.alert_icu<=52)&(df.random_flag=='1')].pat_enc_csn_id_coded.nunique()/df[(df.random_flag=='1')].pat_enc_csn_id_coded.nunique()*100}")
print(f"Control:{df[(df.alert_icu<=52)&(df.random_flag=='2')].pat_enc_csn_id_coded.nunique()/df[(df.random_flag=='2')].pat_enc_csn_id_coded.nunique()*100}")

Treatment:7.443365695792881
Control:11.65644171779141


### Chi squered test

In [19]:
df['icuvisit_within52hours']=0
df.loc[df.alert_icu<=52,'icuvisit_within52hours']=1

In [20]:
import pandas as pd
from scipy.stats import chi2_contingency

# Create a 2x2 contingency table
contingency_table = pd.crosstab(df['random_flag'], df['icuvisit_within52hours'].fillna(0))

# Run Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Square Test")
print(f"Chi2 statistic: {chi2:.3f}")
print(f"Degrees of freedom: {dof}")
print(f"P-value: {p:.4f}")
print("Expected frequencies:")
print(expected)

Chi-Square Test
Chi2 statistic: 6.872
Degrees of freedom: 1
P-value: 0.0088
Expected frequencies:
[[271.21828104  69.78171896]
 [311.78171896  80.21828104]]


## Randomized patient who have been transfered to ICU (Encounter Level)

In [21]:
%%bigquery df
WITH random_flag AS (
  SELECT anon_id,
   pat_enc_csn_id_coded, 
   smrtdta_elem_value AS random_flag 
   FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_smrtdta` 
   WHERE concept_id = 'SHC#6051'
),
 
discharges_selected AS (
  SELECT anon_id, pat_enc_csn_id_coded, effective_time_jittered, department_name FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt`
    INNER JOIN `som-nero-phi-jonc101-secure.shc_core_updates.shc_dep_map` USING(department_id)
    INNER JOIN `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-04-15` USING (anon_id)
    WHERE event_type = 'Discharge' AND UPPER(department_name) LIKE ANY ('B3', 'C3', 'M7', 'L7', '1%WEST%', '2%NORTH%', '2%WEST%', '3%WEST%')
    AND effective_time_jittered - INTERVAL jitter DAY BETWEEN '2024-08-15' AND '2025-03-15'
),
 
admissions AS (
  SELECT anon_id, pat_enc_csn_id_coded, effective_time_jittered FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt` WHERE event_type = 'Admission'
),
 
hospital_stays_selected AS (
  SELECT anon_id, pat_enc_csn_id_coded, adm.effective_time_jittered AS adm_time, dis.effective_time_jittered AS dis_time, department_name,
    DATETIME_DIFF(dis.effective_time_jittered, adm.effective_time_jittered, DAY)+1 AS duration, --for normalization
    DATETIME_DIFF(dis.effective_time_jittered, adm.effective_time_jittered, MINUTE)/60 AS los_hours --for secondary outcome
  FROM admissions adm INNER JOIN discharges_selected dis USING (anon_id, pat_enc_csn_id_coded)
),
study as (
select random_flag.*,
    hospital_stays_selected.adm_time,
    hospital_stays_selected.dis_time,
    hospital_stays_selected.duration,
    hospital_stays_selected.los_hours 
from random_flag 
inner join hospital_stays_selected 
using(anon_id, pat_enc_csn_id_coded)
),
transfer_icu as (
SELECT *,
1 as ICUvisit
FROM `som-nero-phi-jonc101.shc_core_2024.adt`
WHERE event_type = 'Transfer In' 
     AND pat_lv_of_care = 'Critical Care'
)
select c.anon_id,
c.pat_enc_csn_id_coded,
t.effective_time_jittered,
t.ICUvisit,  
c.random_flag,
from study c 
left join transfer_icu t using (anon_id,pat_enc_csn_id_coded)

Query is running:   0%|          |

Downloading:   0%|          |

In [26]:
print(f"Treatment:{df[(df.ICUvisit==1)&(df.random_flag=='1')].pat_enc_csn_id_coded.nunique()/df[(df.random_flag=='1')].pat_enc_csn_id_coded.nunique()*100}")
print(f"Control:{df[(df.ICUvisit==1)&(df.random_flag=='2')].pat_enc_csn_id_coded.nunique()/df[(df.random_flag=='2')].pat_enc_csn_id_coded.nunique()*100}")

Treatment:7.556620209059234
Control:8.123129542539546


### Test the P-Value using chisquerd test

In [57]:
import pandas as pd
from scipy.stats import chi2_contingency

# Create a 2x2 contingency table
contingency_table = pd.crosstab(df['random_flag'], df['ICUvisit'].fillna(0))

# Run Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-Square Test")
print(f"Chi2 statistic: {chi2:.3f}")
print(f"Degrees of freedom: {dof}")
print(f"P-value: {p:.4f}")
print("Expected frequencies:")
print(expected)

Chi-Square Test
Chi2 statistic: 3.722
Degrees of freedom: 1
P-value: 0.0537
Expected frequencies:
[[4208.88775209  799.11224791]
 [4334.11224791  822.88775209]]


## Demographic of randomized patient

In [3]:
%%bigquery df
WITH random_flag AS (
  SELECT anon_id,
   pat_enc_csn_id_coded, 
   smrtdta_elem_value AS random_flag 
   FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_smrtdta` 
   WHERE concept_id = 'SHC#6051'
),
 
discharges_selected AS (
  SELECT anon_id, pat_enc_csn_id_coded, effective_time_jittered, department_name FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt`
    INNER JOIN `som-nero-phi-jonc101-secure.shc_core_updates.shc_dep_map` USING(department_id)
    INNER JOIN `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-04-15` USING (anon_id)
    WHERE event_type = 'Discharge' AND UPPER(department_name) LIKE ANY ('B3', 'C3', 'M7', 'L7', '1%WEST%', '2%NORTH%', '2%WEST%', '3%WEST%')
    AND effective_time_jittered - INTERVAL jitter DAY BETWEEN '2024-08-15' AND '2025-03-15'
),
 
admissions AS (
  SELECT anon_id, pat_enc_csn_id_coded, effective_time_jittered FROM `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt` WHERE event_type = 'Admission'
),
 
hospital_stays_selected AS (
  SELECT anon_id, pat_enc_csn_id_coded, adm.effective_time_jittered AS adm_time, dis.effective_time_jittered AS dis_time, department_name,
    DATETIME_DIFF(dis.effective_time_jittered, adm.effective_time_jittered, DAY)+1 AS duration, --for normalization
    DATETIME_DIFF(dis.effective_time_jittered, adm.effective_time_jittered, MINUTE)/60 AS los_hours --for secondary outcome
  FROM admissions adm INNER JOIN discharges_selected dis USING (anon_id, pat_enc_csn_id_coded)
),
study as (
select random_flag.*,hospital_stays_selected.adm_time,hospital_stays_selected.duration,hospital_stays_selected.los_hours 
from random_flag 
inner join hospital_stays_selected 
using(anon_id, pat_enc_csn_id_coded)
),
demo as (
  select c.* ,
  DATETIME_DIFF(c.adm_time,demo.birth_date_jittered,YEAR) as Age,
  demo.gender,
  demo.canonical_race
  FROM study  c inner join `som-nero-phi-jonc101-secure.shc_core_updates.shc_demographic` demo
  using(anon_id)
) 
select * from demo

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
df['Age']=df['Age'].astype(int)
print(f"Contorl:{df[df.random_flag=='2'].Age.quantile(0.5),df[df.random_flag=='2'].Age.quantile(0.25),df[df.random_flag=='2'].Age.quantile(0.75)}")
print(f"Treatment:{df[df.random_flag=='1'].Age.quantile(0.5),df[df.random_flag=='1'].Age.quantile(0.25),df[df.random_flag=='1'].Age.quantile(0.75)}")
print(df[(df.canonical_race=='Unknown')&(df.random_flag=='2')][['anon_id','pat_enc_csn_id_coded']].drop_duplicates().shape[0]/df[df.random_flag=='2'][['anon_id','pat_enc_csn_id_coded']].drop_duplicates().shape[0]*100)


Contorl:(70.0, 57.0, 80.0)
Treatment:(69.0, 56.0, 80.0)
2.3728088926891835


In [20]:
%%bigquery lab_orders_in_units
with base as (
select anon_id,
pat_enc_csn_id_coded,
smrtdta_elem_value,
from `som-nero-phi-jonc101-secure.shc_core_updates.shc_smrtdta`
where concept_id like 'SHC#6051'
and smrtdta_elem_value in ('1','2')
),
department as (
  select anon_id,
  pat_enc_csn_id_coded,
  department_name,
  effective_time_jittered,
  from `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt`
  inner join `som-nero-phi-jonc101-secure.shc_core_updates.shc_dep_map`
  using(department_id)
  where event_type='Discharge'
and department_name like any ('B3','C3','M7','L7','1%West',
'2%North', '2%West', '3%West')
and effective_time_jittered>='2024-08-15'
and effective_time_jittered<='2025-03-15'
),
study_cohort as (
select c.*,
from base  c
inner join department d using (anon_id,pat_enc_csn_id_coded)
),
lab_taken as (
select lr.anon_id,
lr.pat_enc_csn_id_coded,
lr.order_id_coded,
lr.taken_time_jittered,
#lr.base_name,
lr.group_lab_name,
op.department_id
from  `som-nero-phi-jonc101-secure.shc_core_updates.shc_lab_result` lr
inner join  `som-nero-phi-jonc101-secure.shc_core_updates.shc_order_proc` op
on lr.anon_id=op.anon_id and lr.pat_enc_csn_id_coded=op.pat_enc_csn_id_coded and lr.order_id_coded=op.order_proc_id_coded
#where lower(base_name) in ('wbc','hgb','plt')
where lower(group_lab_name) like '%cbc%dif%'
),
lab_taken_in_unit as (
select lr.anon_id,
lr.pat_enc_csn_id_coded,
lr.order_id_coded,
lr.taken_time_jittered,
#lr.base_name,
lr.group_lab_name
from lab_taken lr
inner join  `som-nero-phi-jonc101-secure.shc_core_updates.shc_dep_map` using(department_id)
where department_name like any ('B3','C3','M7','L7','1%West',
'2%North', '2%West', '3%West')
group by anon_id,pat_enc_csn_id_coded,order_id_coded,taken_time_jittered,group_lab_name
order by anon_id,pat_enc_csn_id_coded,order_id_coded,taken_time_jittered,group_lab_name
)
select c.*,
lr.taken_time_jittered,
lr.order_id_coded,
#lr.base_name,
lr.group_lab_name,
from study_cohort c
inner join lab_taken_in_unit lr using (anon_id,pat_enc_csn_id_coded)


Query is running:   0%|          |

Downloading:   0%|          |

In [37]:
lab_orders_in_units['taken_time_jittered'] = pd.to_datetime(lab_orders_in_units['taken_time_jittered'])
lab_orders_in_units['taken_date_jittered'] = lab_orders_in_units['taken_time_jittered'].dt.date
group1 = lab_orders_in_units[lab_orders_in_units['smrtdta_elem_value'] == '2']
te=group1.groupby(['anon_id', 'pat_enc_csn_id_coded', 'smrtdta_elem_value', 'taken_date_jittered'],as_index=False).count()
te.taken_time_jittered.mean()

1.0513100436681222

In [48]:
# Corrected datetime conversion
lab_orders_in_units['taken_time_jittered'] = pd.to_datetime(lab_orders_in_units['taken_time_jittered'])
lab_orders_in_units['taken_date_jittered'] = lab_orders_in_units['taken_time_jittered'].dt.date

# Separate the groups
group1 = lab_orders_in_units[lab_orders_in_units['smrtdta_elem_value'] == '1']
group2 = lab_orders_in_units[lab_orders_in_units['smrtdta_elem_value'] == '2']

# Group by multiple columns (must be a list)
te = group1.groupby(['anon_id', 'pat_enc_csn_id_coded', 'smrtdta_elem_value', 'taken_date_jittered'], as_index=False).count()
te2 = group2.groupby(['anon_id', 'pat_enc_csn_id_coded', 'smrtdta_elem_value', 'taken_date_jittered'], as_index=False).count()

# Calculate the average number of 'base_name' occurrences
avg_base_name = te['taken_time_jittered'].mean()#.quantile(0.25)
avg_base_name2 = te2['taken_time_jittered'].mean()#quantile(0.25)
print(f'Group 1, average number of base_name: {avg_base_name}')
print(f'Group 2, average number of base_name: {avg_base_name2}')

Group 1, average number of base_name: 1.062948779320249
Group 2, average number of base_name: 1.0513100436681222


In [63]:
%%bigquery lab_orders
with base as (
select anon_id,
pat_enc_csn_id_coded,
smrtdta_elem_value,
from `som-nero-phi-jonc101-secure.shc_core_updates.shc_smrtdta`
where concept_id like 'SHC#6051'
and smrtdta_elem_value in ('1','2')
),
department as (
  select anon_id,
  pat_enc_csn_id_coded,
  department_name,
  effective_time_jittered,
  from `som-nero-phi-jonc101-secure.shc_core_updates.shc_adt`
  inner join `som-nero-phi-jonc101-secure.shc_core_updates.shc_dep_map`
  using(department_id)
  where event_type='Discharge'
and department_name like any ('B3','C3','M7','L7','1%West',
'2%North', '2%West', '3%West')
and effective_time_jittered>='2024-08-15'
and effective_time_jittered<='2025-03-15'
),
study_cohort as (
select c.*,
from base  c
inner join department d using (anon_id,pat_enc_csn_id_coded)
),
lab_taken as (
select lr.anon_id,
lr.pat_enc_csn_id_coded,
lr.order_id_coded,
lr.taken_time_jittered,
#r.base_name,
'cbc' as group_lab_name,
#op.department_id
from  `som-nero-phi-jonc101-secure.shc_core_updates.shc_lab_result` lr
inner join  `som-nero-phi-jonc101-secure.shc_core_updates.shc_order_proc` op
on lr.anon_id=op.anon_id and lr.pat_enc_csn_id_coded=op.pat_enc_csn_id_coded and lr.order_id_coded=op.order_proc_id_coded
where 
#lower(base_name) in ('wbc','hgb','plt')
#and
lower(group_lab_name) like '%cbc%dif%'
group by anon_id,pat_enc_csn_id_coded,order_id_coded,taken_time_jittered,group_lab_name
order by anon_id,pat_enc_csn_id_coded,order_id_coded,taken_time_jittered,group_lab_name

)
select c.*,
lr.taken_time_jittered,
#lr.base_name,
lr.group_lab_name,
from study_cohort c
left join lab_taken lr using (anon_id,pat_enc_csn_id_coded)

Query is running:   0%|          |

Downloading:   0%|          |

In [71]:
lab_orders[(lab_orders.taken_time_jittered.isna())&(lab_orders.smrtdta_elem_value=='2')][['anon_id','pat_enc_csn_id_coded']].drop_duplicates().shape[0]/lab_orders[(lab_orders.smrtdta_elem_value=='2')][['anon_id','pat_enc_csn_id_coded']].drop_duplicates().shape[0]*100

27.886497064579256

In [76]:
group1[group1.taken_date_jittered.isna()]

,anon_id,pat_enc_csn_id_coded,smrtdta_elem_value,taken_time_jittered,group_lab_name,taken_date_jittered
0,JC6517739,131381240520.000000000,1,NaT,None,NaT
1,JC2717494,131392677712.000000000,1,NaT,None,NaT
6,JC2463780,131391067126.000000000,1,NaT,None,NaT
41,JC3797487,131398733799.000000000,1,NaT,None,NaT
42,JC1833229,131378654763.000000000,1,NaT,None,NaT
...,...,...,...,...,...,...
15274,JC3564132,131374079282.000000000,1,NaT,None,NaT
15326,JC1524567,131389306557.000000000,1,NaT,None,NaT
15366,JC2893351,131370520665.000000000,1,NaT,None,NaT
15389,JC1165911,131384926348.000000000,1,NaT,None,NaT


In [89]:
group1[(group1.anon_id=='JC6345733')&(group1.pat_enc_csn_id_coded==131379875842)&(group1.taken_date_jittered==pd.Date('2024-06-18'))]
#te[te.taken_time_jittered>6]

AttributeError: module 'pandas' has no attribute 'date'

In [73]:
# Corrected datetime conversion
lab_orders['taken_time_jittered'] = pd.to_datetime(lab_orders['taken_time_jittered'])
lab_orders['taken_date_jittered'] = lab_orders['taken_time_jittered'].dt.date

# Separate the groups
group1 = lab_orders[lab_orders['smrtdta_elem_value'] == '1']
group2 = lab_orders[lab_orders['smrtdta_elem_value'] == '2']

# Group by multiple columns (must be a list)
te = group1.groupby(['anon_id', 'pat_enc_csn_id_coded', 'smrtdta_elem_value', 'taken_date_jittered'], as_index=False).count()
te2 = group2.groupby(['anon_id', 'pat_enc_csn_id_coded', 'smrtdta_elem_value', 'taken_date_jittered'], as_index=False).count()

# Calculate the average number of 'base_name' occurrences
avg_base_name = te['taken_time_jittered'].max()
avg_base_name2 = te2['taken_time_jittered'].max()
print(f'Group 1, average number of base_name: {avg_base_name}')
print(f'Group 2, average number of base_name: {avg_base_name2}')

Group 1, average number of base_name: 7
Group 2, average number of base_name: 6
